In [11]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('disneyRideTimes.csv')

/var/folders/h1/tr1q_6210b3fh7z5w19c2qvm0000gp/T/ipykernel_16289/4262990351.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('disneyRideTimes.csv')


In [21]:
df.dtypes

rideID            int64
dateTime         object
waitMins        float64
fastPassTime     object
status           object
dtype: object

In [29]:
df[(df['rideID'] == 353437) & (df['status'] == 'Operating')][-50:]

,rideID,dateTime,waitMins,fastPassTime,status
1369985,353437,2018-09-13 16:00:14,50.0,18:55:00,Operating
1369986,353437,2018-09-13 16:30:05,50.0,19:25:00,Operating
1369987,353437,2018-09-13 17:00:15,40.0,19:50:00,Operating
1369988,353437,2018-09-13 17:30:04,40.0,20:10:00,Operating
1369989,353437,2018-09-13 18:00:14,60.0,20:25:00,Operating
1369990,353437,2018-09-13 18:30:05,75.0,NaN,Operating
1369991,353437,2018-09-13 19:00:14,40.0,NaN,Operating
1369994,353437,2018-09-13 20:30:04,10.0,NaN,Operating
1369995,353437,2018-09-13 21:00:12,5.0,NaN,Operating
1370020,353437,2018-09-14 09:30:04,5.0,09:35:00,Operating
